In [1]:
import pandas as pd
import requests
import numpy as np

In [2]:
reg = pd.read_csv('conso-elec-gaz-region.csv', sep=';')
dep = pd.read_csv('conso-elec-gaz-departement.csv', sep=';')
com = pd.read_csv('conso-elec-gaz-commune.csv', sep=';')

/tmp/ipykernel_296017/733422901.py:3: DtypeWarning: Columns (25,27,29) have mixed types. Specify dtype option on import or set low_memory=False.
  com = pd.read_csv('conso-elec-gaz-commune.csv', sep=';')


In [7]:
com.shape

(501565, 36)

In [19]:
com.code_commune

0         65105
1         65107
2         65109
3         65111
4         65114
          ...  
501560    72322
501561    72325
501562    72330
501563    72336
501564    72338
Name: code_commune, Length: 501565, dtype: object

In [11]:
com.head()
com.columns

Index(['operateur', 'annee', 'filiere', 'consoa', 'pdla',
       'nombre_maille_secretisees_a', 'indquala', 'consoi', 'pdli',
       'nombre_maille_secretisees_i', 'indquali', 'consot', 'pdlt',
       'nombre_maille_secretisees_t', 'indqualt', 'consor', 'pdlr',
       'nombre_maille_secretisees_r', 'indqualr', 'thermor', 'partr',
       'consona', 'pdlna', 'nombre_maille_secretisees_na', 'indqualna',
       'code_commune', 'libelle_commune', 'code_epci', 'libelle_epci',
       'code_departement', 'libelle_departement', 'code_region',
       'libelle_region', 'id_filiere', 'consototale', 'code_postal'],
      dtype='object')

In [9]:
len(com['libelle_commune'].unique())

32742

In [3]:
url = 'https://geo.api.gouv.fr/communes/65105?fields=nom,code,codesPostaux,siren,codeEpci,codeDepartement,codeRegion,population&format=geojson&geometry=centre'
response = requests.get(url)
print("Statut de la réponse :", response.status_code)
response.text

Statut de la réponse : 200


'{"type":"Feature","properties":{"nom":"Bourg-de-Bigorre","code":"65105","codesPostaux":["65130"],"siren":"216501056","codeEpci":"200070787","codeDepartement":"65","codeRegion":"76","population":196},"geometry":{"type":"Point","coordinates":[0.2571,43.0846]}}'

In [4]:
def get_url(code_commune):
    url = f'https://geo.api.gouv.fr/communes/{code_commune}?fields=nom,code,codesPostaux,siren,codeEpci,codeDepartement,codeRegion,population&format=geojson&geometry=centre'
    return url

def get_coordinates(com):

    unique_codes = com['code_commune'].unique()
    coordinates_dict = {}

    total = len(unique_codes)

    for i, code in enumerate(unique_codes, 1):

        url = get_url(code)
        response = requests.get(url)
        
        print(f"Traitement du code de commune {i} sur {total}: {code}, statut de la réponse: {response.status_code}")

        if response.status_code == 200:
            data = response.json()
            coordinates = data['geometry']['coordinates']
            coordinates_dict[code] = coordinates
        else:
            print(f"Erreur lors de la récupération des coordonnées pour le code de commune {code}")
            coordinates_dict[code] = [np.nan, np.nan]

    return coordinates_dict
    

In [6]:
coordinates_dict = get_coordinates(com)

Traitement du code de commune 1 sur 68070: 65105, statut de la réponse: 200
Traitement du code de commune 2 sur 68070: 65107, statut de la réponse: 200
Traitement du code de commune 3 sur 68070: 65109, statut de la réponse: 200
Traitement du code de commune 4 sur 68070: 65111, statut de la réponse: 200
Traitement du code de commune 5 sur 68070: 65114, statut de la réponse: 200
Traitement du code de commune 6 sur 68070: 65115, statut de la réponse: 200
Traitement du code de commune 7 sur 68070: 65117, statut de la réponse: 200
Traitement du code de commune 8 sur 68070: 65125, statut de la réponse: 200
Traitement du code de commune 9 sur 68070: 65127, statut de la réponse: 200
Traitement du code de commune 10 sur 68070: 65128, statut de la réponse: 200
Traitement du code de commune 11 sur 68070: 65129, statut de la réponse: 200
Traitement du code de commune 12 sur 68070: 65133, statut de la réponse: 200
Traitement du code de commune 13 sur 68070: 65149, statut de la réponse: 200
Traiteme

In [ ]:
def df_lon_lat(com, coordinates_dict):

    com['lon'] = com['code_commune'].map(coordinates_dict).apply(lambda x: x[0])
    com['lat'] = com['code_commune'].map(coordinates_dict).apply(lambda x: x[1])
        
    return com

new_com = df_lon_lat(com, coordinates_dict)
new_com